### Find best perfoming model

below a search for the best model is performed.
For every time the notebook is runned following a knn, SVM and a FR model has been trained. 
Settings that have been adjested every run:
        <ol>
        <li> normalize scaling on PCA = 0.9 (PCA90 in filename) </li>
        <li> standardize scaling on PCA = 0.9 (PCA90 in filename) </li>
        <li> standardize scaling on PCA = 0.999 (PCA999 in filename) </li>
        <li> standardize scaling on PCA = 0.8 (PCA80 in filename) </li>
        <li> standardize scaling PCA is commented (PCA1 in filename) </li>
        </ol>
markdowns have been placed above the positions of the adjustments
Metrics included: 

#### Please scroll through the notebook first, adjust the needed parameters and run it

    

In [ ]:
import numpy as np
import pandas as pd
import MachineLearning as ML # own functions

In [ ]:
smiles = pd.read_csv('tested_molecules-1.csv')
smiles = smiles['SMILES']

In [ ]:
allAHDL1inhibitors = ML.importFiles(nrFiles=2)
ML.writeNewMolfile(allAHDL1inhibitors)

In [ ]:
allDescrs = ML.createDescriptorDf()
# allDescrs = allDescrs.drop(columns=["SMILES"])
x = ML.generateMorganFingerprint()
y = allAHDL1inhibitors[1][1:].astype(int)
sum(y)/len(y)
x, xDims = ML.CombineDescriptorsAndFigerprints(x,allDescrs)


### input needed
next cell 'standardize' or 'normalize' can be inserted

As well as the 'value' of PCAfeatureReduction(xScale, 'value'=int)

In [ ]:
xScale = ML.scaleData(x,scaletype='standardize')
# xPrep, _ = ML.PCAfeatureReduction(xScale, 0.999)
xPrep = xScale
xTrain, xTest, yTrain, yTest, cv = ML.splitTrainTestData(xPrep,y,
                                                         printSplit=True)

### RFC model

In [ ]:
if 'SMILES' in xTrain:
    xTrain = xTrain.drop(columns=["SMILES"])
RFmodel, bestParam, RFgrid, param_grid, BestScoreRFgrid = ML.trainRF(xTrain, yTrain, cv)



## Do not forget to change: 
'RandomForestClassifier_PCAVALUE_SCALINGTYPE' in ML.saveTrainedModel

For instance when in ML.PCAfeatureReduction(xScale, 0.9) and in ML.scaleData(x,scaletype='standardize')

 PCAVALUE = '90' and SCALINGTYPE = 'std'

In [ ]:
if 'SMILES' in xTest:
    xTest = xTest.drop(columns=["SMILES"])
    
pred, balAcc, evaluationReport, confusionMatrix, predProb = ML.testTrainedModel(xTest,
                                                                                 yTest, 
                                                                                 model=RFmodel, 
                                                                                 savedModelfilename=None, 
                                                                                 scaledData=True, 
                                                                                 scaledDatafile=None)
print(pred,'\n', 'Balanced Accuracy:  ', balAcc,'\n', evaluationReport, '\n', confusionMatrix, '\n', predProb)
ML.saveTrainedModel(RFmodel, 'RandomForestClassifier_PCA1_std') # optional

In [ ]:
ThreshPreds, threshAcc, coverage= ML.thresholdedAccuracy(yTest, pred, predProb, threshold=0.7)
print(threshAcc, '\n', len(ThreshPreds))

### SVC model

In [ ]:
SVCmodel, bestParam, RFgrid, param_grid, BestScoreRFgrid = ML.trainSVC(xTrain, yTrain, cv)

## Do not forget to change: 
'SupportVectorClassifier_PCAVALUE_SCALINGTYPE' in ML.saveTrainedModel

For instance when in ML.PCAfeatureReduction(xScale, 0.9) and in ML.scaleData(x,scaletype='standardize')

 PCAVALUE = '90' and SCALINGTYPE = 'std'

In [ ]:
pred, balAcc, evaluationReport, confusionMatrix, predProb = ML.testTrainedModel(xTest, 
                                                                                yTest, 
                                                                                model=SVCmodel, 
                                                                                savedModelfilename=None, 
                                                                                scaledData=True, 
                                                                                scaledDatafile=None)

print(pred,'\n', 'Balanced Accuracy:  ', balAcc,'\n', evaluationReport, '\n', confusionMatrix, '\n', predProb)
ML.saveTrainedModel(SVCmodel, 'SupportVectorClassifier_PCA1_std') # optional

In [ ]:
ThreshPreds, threshAcc, coverage= ML.thresholdedAccuracy(yTest, pred, predProb, threshold=0.7)
print(threshAcc, '\n', len(ThreshPreds))

In [ ]:
knnModel, bestParam, RFgrid, param_grid, BestScoreRFgrid = ML.trainKnn(xTrain, 
                                                                       yTrain, 
                                                                       cv, 
                                                                       list(range(1,31)))


## Do not forget to change: 
'KnnClassifier_PCAVALUE_SCALINGTYPE' in ML.saveTrainedModel

For instance when in ML.PCAfeatureReduction(xScale, 0.9) and in ML.scaleData(x,scaletype='standardize')

 PCAVALUE = '90' and SCALINGTYPE = 'std'

In [ ]:
pred, balAcc, evaluationReport, confusionMatrix, predProb = ML.testTrainedModel(xTest, 
                                                                                yTest, 
                                                                                model=knnModel, 
                                                                                savedModelfilename=None, 
                                                                                scaledData=True, 
                                                                                scaledDatafile=None)

print(pred,'\n', 'Balanced Accuracy:  ', balAcc,'\n', evaluationReport, '\n', confusionMatrix, '\n', predProb)
ML.saveTrainedModel(knnModel, 'KnnClassifier_PCA1_std') # optional

In [ ]:
ThreshPreds, threshAcc, coverage= ML.thresholdedAccuracy(yTest, pred, predProb, threshold=0.7)
print(threshAcc, '\n', len(ThreshPreds))

#### when the best model is found: save the predictions (first 100)

Best performing model is the Random Forest model with PCA 0.999 and standardized scaling 

predicting the top 100 molucules below:

In [ ]:

top100MolsWithProbs = ML.top100molecules('RandomForestClassifier_PCA999_std.pkl', 0.999)
top100Mols = top100MolsWithProbs['SMILES']
top100Mols.to_csv('top100UntestedMolecules.csv', index=False)